In [110]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    'BX-Books.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [92]:
# We get books with at least 100 ratings
value_counts_books = df_ratings['isbn'].value_counts()
good_books = value_counts_books[value_counts_books >= 100].index

# We get users with at least 200 ratings
value_counts_users = df_ratings['user'].value_counts()
good_users = value_counts_users[value_counts_users >= 200].index

# We create the dataframe with the good books
df_good_books = df_books.copy()
df_good_books = df_books[df_books['isbn'].isin(good_books)]

# We create the dataframe with the good ratings
df_good_ratings = df_ratings.copy()
df_good_ratings = df_ratings[df_ratings['user'].isin(good_users)]

# We merge both of the dataframes on isbn
df_final = pd.merge(left=df_good_books, right=df_good_ratings, on='isbn')

In [118]:
# We create a pivot table that will be passed to the Nearest Neighbors algorithm
user_book_ratings = df_final.pivot_table(index='title', columns='user', values='rating')
user_book_ratings.fillna(0, inplace=True)  # Fill missing values with 0s

user_book_ratings

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [221]:
# Finally, we create the model using the cosine metric required in the tests
model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='cosine')
model.fit(user_book_ratings)

NearestNeighbors(metric='cosine')

In [222]:
# function to return recommended books - this will be tested

# We get the 4 nearest neighbors from the book title and put their distances and titles in the recommended_books array
def get_recommends(book = ""):
  recommended_books = []
  dist, index = model.kneighbors([user_book_ratings.loc[book]])

  for i in range(4,1,-1):
    recommended_books.append([user_book_ratings.iloc[index[0][i]].name, round(dist[0][i],2)])
  return [book, recommended_books]

In [223]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77]]]
You passed the challenge! 🎉🎉🎉🎉🎉
